In [22]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

Для доверительных интервалов, асимптотическая оценка ищется из предположений нормальности:
$$\dfrac{g(n)(\hat{\theta}-\theta)}{\sigma(\theta)}\sim N(0,1)$$
Если $\hat{\theta}$ -- состоятельная оценка $\theta$ (то есть имеется сходимость оценки к параметру), то имеет место замена:
$$\dfrac{g(n)(\hat{\theta}-\theta)}{\sigma(\hat{\theta})}\sim N(0,1)$$
Что именно изменилось: дисперсия $\sigma$ в знаменателе теперь зависит от оценки $\hat{\theta}$, а не от самой $\theta$. Это дает нам возможность сделать, например, такой переход: Взять состоятельную оценку $\rightarrow$ оценить $\sigma$ выборочной дисперсией $S^2$ $\rightarrow$ выразить $S^2$ через $\hat{\theta}$.
И тогда ас. доверительный интервал точности $1-\alpha$ будет иметь вид:
$$\left( \hat{\theta} - \dfrac{u_{1-\alpha/2}\sigma(\hat{\theta})}{g(n)},  \hat{\theta} + \dfrac{u_{1-\alpha/2}\sigma(\hat{\theta})}{g(n)}\right)$$
Здесь $u_x$ - квантиль уровня $x$ для нормального распределения $N(0,1)$
Зачастую в качестве $g(n)$ можно брать $\sqrt{n}$

# I. Про детей
Не знаю, где найти данные. Но предположим, у нас есть массив, где 0 означает девочка, 1 - мальчик. То есть имеем распределение Бернулли и хотим оценить параметр этого распределения $p$

In [35]:
p = 0.3
N = 100

x = np.random.binomial(1, p, N)
x

array([0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

Состоятельной оценкой для $p$ будет $\hat{p} = \overline{X}$. $\sigma(p) = \sqrt{p(1-p)}$ - обычная формула дисперсии. Тепер, так как мы можем замеить $\sigma(p)$ на $\sigma(\hat{p})$, то есть на $\sqrt{\overline{X}(1-\overline{X})}$. Тогда получим:
$$\dfrac{\sqrt{n}(\overline{x}-p)}{\sqrt{\overline{X}(1-\overline{X})}}\sim N(0,1)$$
Отсюда, асимптотический доверительный интервал:
$$\left(\overline{X} - \dfrac{u_{1-\alpha/2}\sqrt{\overline{X}(1-\overline{X})}}{\sqrt{n}},\overline{X} + \dfrac{u_{1-\alpha/2}\sqrt{\overline{X}(1-\overline{X})}}{\sqrt{n}}\right)$$
Традиционно $\alpha = 0.05$

In [36]:
alpha = 0.05

u = scipy.stats.norm.ppf(1-alpha/2)
x_mean = x.mean()
ce_l = x_mean - u*np.sqrt(x_mean*(1-x_mean))/np.sqrt(N)
ce_r = x_mean + u*np.sqrt(x_mean*(1-x_mean))/np.sqrt(N)
ce_l, ce_r

(0.23784000079994191, 0.4221599992000581)

Вот такой вот интервал, наш параметр даже в него попал. Можно поварьировать $N$ и увидеть, как он стягивается

А как построить точный интервал я не придумал :(

# II.

Данных снова нет, снова насемплим их из какого-то нормального распределения с погрешностями

In [68]:
N = 100
x = np.random.normal(300, 50, N) + np.absolute(np.random.normal(3, 3, N))

Интервал оценки среднего через распределение Стьюдента имеет вид:
$$\left(\overline{X} - t_{\alpha/2, n-1}\frac{S_n}{\sqrt{n}}, \overline{X} + t_{\alpha/2, n-1}\frac{S_n}{\sqrt{n}}\right)$$
Здесь $t_{\alpha,n-1}$ - квантиль уровня $\alpha$ распределения Стьюдента с $n-1$ степенью свободы, $S_n$ - корень из выборочной дисперсии

In [69]:
x_mean = x.mean()
t = scipy.stats.t.ppf(1-0.05/2, N-1)
std = np.sqrt(x.std())
ce_l = x_mean - t*std/np.sqrt(N)
ce_r = x_mean + t*std/np.sqrt(N)
ce_l, ce_r

(295.9514821197486, 298.6431190190411)

На основе нормального распределения: $\overline{X}$ будет состоятельной оценкой, $\sigma(\theta)$ оценим как $\sqrt{S^2}$, тогда искомый интервал:

$$\left( \overline{X} - \dfrac{u_{1-\alpha/2}\sqrt{S^2}}{\sqrt{n}},\overline{X} + \dfrac{u_{1-\alpha/2}\sqrt{S^2}}{\sqrt{n}}\right)$$

In [70]:
alpha = 0.05

x_mean = x.mean()
x_std = np.sqrt(x.std())
u = scipy.stats.norm.ppf(1-alpha/2)

ce_l = x_mean - u*x_std/np.sqrt(N)
ce_r = x_mean + u*x_std/np.sqrt(N)
ce_l, ce_r

(295.96793197941753, 298.62666915937217)

Теперь бутстреп (что такое опция normal я не знаю):

In [82]:
ans = scipy.stats.bootstrap((x,), np.mean, n_resamples=1000, random_state = 13)
interval = ans.confidence_interval
interval.low, interval.high

(288.0683492772335, 305.93754586434756)

Какой-то вывод: Бутстреп дал самый длинный интервал, стьюдент и нормальный - более-менее одинаковый